Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

## S23. De los negocios que tienen acceso para silla de ruedas (si no dice nada asuma que no tiene) y tienen más de 10 reviews, ¿cuál es el mejor? ¿y el peor?

### Installs e imports

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 63.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a6419f45ba09c1f86ea11dbd571b2425cbd32af79056f48fd4029a9135fa1e0f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After t

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

### Resolución

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('/content/drive/MyDrive/Datos/TP2/business_parquet', inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd.take(5)

[Row(business_id='Pns2l4eNsfO8kk83dixA6A', name='Abby Rappoport, LAC, CMQ', address='1616 Chapala St, Ste 2', city='Santa Barbara', state='CA', postal_code='93101', latitude=34.4266787, longitude=-119.7111968, stars=5.0, review_count=0.0, is_open=0, attributes="{'ByAppointmentOnly': 'True'}", categories='Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists', hours=None),
 Row(business_id='mpf3x-BjTdTEA3yCZrAYPw', name='The UPS Store', address='87 Grasso Plaza Shopping Center', city='Affton', state='MO', postal_code='63123', latitude=38.551126, longitude=-90.335695, stars=3.0, review_count=8.0, is_open=1, attributes="{'BusinessAcceptsCreditCards': 'True'}", categories='Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services', hours="{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"),
 Row(business_id='tUFrWirKiKi_TAns

Me quedo con los negocios con acceso a silla de ruedas (contienen 'WheelchairAccessible': 'True' en attributes) y tienen más de 10 reviews:

In [ ]:
filtered = rdd.filter(lambda x: x.attributes and "'WheelchairAccessible\': \'True\'" in x.attributes and x.review_count > 10).cache()
filtered.first()

Row(business_id='tUFrWirKiKi_TAnsVWINQQ', name='Target', address='5255 E Broadway Blvd', city='Tucson', state='AZ', postal_code='85711', latitude=32.223236, longitude=-110.880452, stars=3.5, review_count=11.0, is_open=0, attributes='{\'BikeParking\': \'True\', \'BusinessAcceptsCreditCards\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'CoatCheck\': \'False\', \'RestaurantsTakeOut\': \'False\', \'RestaurantsDelivery\': \'False\', \'Caters\': \'False\', \'WiFi\': "u\'no\'", \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'WheelchairAccessible\': \'True\', \'HappyHour\': \'False\', \'OutdoorSeating\': \'False\', \'HasTV\': \'False\', \'RestaurantsReservations\': \'False\', \'DogsAllowed\': \'False\', \'ByAppointmentOnly\': \'False\'}', categories='Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores', hours="{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday

Obtengo el mejor basandome en su cantidad de estrellas:

In [ ]:
filtered.reduce(lambda x, y: x if x.stars > y.stars else y)

Row(business_id='izSgTrqebu8bN8ONOCs6cQ', name='Oat Bakery', address='5 W Haley St', city='Santa Barbara', state='CA', postal_code='93101', latitude=34.4165478074, longitude=-119.6956258296, stars=5.0, review_count=124.0, is_open=1, attributes='{\'Alcohol\': "u\'none\'", \'HasTV\': \'False\', \'BikeParking\': \'True\', \'Caters\': \'True\', \'BusinessAcceptsBitcoin\': \'True\', \'HappyHour\': \'True\', \'BusinessParking\': "{\'garage\': False, \'street\': True, \'validated\': False, \'lot\': True, \'valet\': False}", \'DogsAllowed\': \'True\', \'OutdoorSeating\': \'None\', \'WheelchairAccessible\': \'True\', \'BusinessAcceptsCreditCards\': \'True\', \'RestaurantsTableService\': \'True\', \'BYOB\': \'False\', \'WiFi\': "u\'no\'", \'RestaurantsDelivery\': \'None\', \'RestaurantsPriceRange2\': \'1\', \'RestaurantsReservations\': \'True\', \'RestaurantsTakeOut\': \'None\'}', categories='Bakeries, Vegan, Specialty Food, Food Delivery Services, Food, Coffee & Tea, Gluten-Free, Farmers Market

Obtengo el peor:

In [ ]:
filtered.reduce(lambda x, y: x if x.stars < y.stars else y)

Row(business_id='MtFOOKu3JXClmUEDOCQKFA', name='Cherry Hill Programs', address='4 E Stow Rd, Ste 1', city='Marlton', state='NJ', postal_code='08053', latitude=39.912606, longitude=-74.93051, stars=1.0, review_count=59.0, is_open=1, attributes="{'RestaurantsPriceRange2': '3', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'None', 'DogsAllowed': 'True', 'WheelchairAccessible': 'True'}", categories='Flowers & Gifts, Shopping Centers, Photographers, Event Planning & Services, Event Photography, Photography Stores & Services, Shopping, Cards & Stationery, Arts & Crafts', hours=None)

De los negocios con acceso para sillas de ruedas, Oat Bakery es el mejor con 5 estrellas y Cherry Hill Programs el peor con 1 estrella. Ambos con la máxima y mínima cantidad de estrellas respectivamente, por lo que puede haber más negocios con esa cantidad de estrellas que cuenten como el mejor y el peor.